In [1]:
# Tin Ho INFO 370 A1
# 10.13.2016
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import html
import re
import csv
import numpy as np

In [2]:
# Lists to be created into dataframe table
firstNames = [] #Column 1 REQUIRED
lastNames = [] #Column 2 REQUIRED
phdUniversity = [] #Column 3 REQUIRED
universityID = [] #Column 4 REQUIRED
phdDepartments = [] #Column 5
phdYears = [] #Column 6
facultyUniversity = [] #Column 7 REQUIRED
facultyUniversityID = [] #Column 8 REQUIRED
facultyDepartments = [] #Column 9 REQUIRED
facultyYearStarted = [] #Column 10
jobTitles = [] #Column 11

# Urls from University of Puget Sound
urlsPS = ['http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/asian-languages-cultures/faculty/'
, 'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/classics/faculty/'
, 'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/environmental-policy/faculty/'
, 'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/french-studies/faculty/'
, 'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/gender-studies/faculty/'
, 'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/german-studies/faculty/'
, 'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/neuroscience/faculty/'
, 'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/religion/faculty/']

# Urls from Illinois Institute of Technology
urlsIIT = ['http://engineering.iit.edu/bme/people/faculty'
,'http://engineering.iit.edu/chbe/people/faculty'
,'http://engineering.iit.edu/ece/people/faculty'
,'http://engineering.iit.edu/mmae/people/faculty']

# Urls I have chosen:
#'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/asian-languages-cultures/faculty/'
#'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/classics/faculty/'
#'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/environmental-policy/faculty/'
#'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/french-studies/faculty/'
#'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/gender-studies/faculty/'
#'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/german-studies/faculty/'
#'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/neuroscience/faculty/'
#'http://www.pugetsound.edu/academics/departments-and-programs/undergraduate/religion/faculty/'
#'http://engineering.iit.edu/bme/people/faculty'
#'http://engineering.iit.edu/chbe/people/faculty'
#'http://engineering.iit.edu/ece/people/faculty'
#'http://engineering.iit.edu/mmae/people/faculty'


In [3]:
# Scrapes University of Puget Sound's website for faculty data
for url in urlsPS:
    # Opens and reads url page
    page = urllib.request.urlopen(url) 
    soup = BeautifulSoup(page.read(), 'lxml')
    #print(soup.prettify())
    
    # Already known Information
    u = "University of Puget Sound"
    uID = 365
    uDepartment = soup.find_all('a', class_='short_5')[-1].getText()
    #print(uDepartment[-1].getText())
    #print(uDepartment)
    
    # Find faculty education information such as phdYear and phdSchool
    education = soup.find_all('td', class_='edu')
    #print(education)
    for edu in education:
        e = edu.ul.find_all('li')#.getText()
        phdSchool = 'N/A'
        phdYear = 'N/A'
        for school in e:
            if 'PhD' in school.getText():
                eduInfo = school.getText().split(', ')
                phdYear = eduInfo[-1]
                phdSchool = eduInfo[eduInfo.index('PhD') + 1]
                if phdSchool == 'University of Washington-Seattle':
                    phdSchool = 'University of Washington'
                #phdSchool = school.getText()
                #print(school.getText())

        phdUniversity.append("" + phdSchool)
        phdYears.append(phdYear)  
    
    # Scrapes and cleans faculty postions
    positions = soup.find_all('td', class_='dept')
    for position in positions:
        job = position.p.getText().split(', ')
        jobTitles.append(job[0].replace('\n', "").replace('\t', ""))

    # Scrapes and cleans faculty names
    names = soup.find_all('td', class_='name')
    #print(names)
    for name in names:
        n = name.h3.getText().split()
        if "Dr." in n[0]:
            del n[0]
        if "." in n[1]:
            del n[1]
        if len(n) == 3:
            n[0] = n[0] + ' ' + n[1]
            del n[1]
            
        # Appends information to lists to be used to create dataframe
        firstNames.append(n[0].upper()) 
        lastNames.append(n[1].upper()) 
        facultyUniversity.append(u) 
        facultyUniversityID.append(uID) 
        facultyDepartments.append(uDepartment) 

In [4]:
# Scrapes Illinois Institute of Technology's website for faculty data
for url in urlsIIT:
    # Opens and reads url page
    page = urllib.request.urlopen(url) 
    soup = BeautifulSoup(page.read(), 'lxml')
    
    # Already known information
    u = "Illinois Institute of Technology"
    uID = 129
    uDepartment = soup.find('div', id='department-header').h2.a.span.getText()
    
    # Finds faculty names and individual urls for additional scraping
    people = soup.find_all('span', class_='field-content')
    for person in people:
        # Checks to see if faculty has Ph.D.
        if 'Ph.D.' in person.getText():
            n = person.getText().split(', ')
            n = n[0].strip()
            fullName = n.split()
            if '.' in fullName[0]:
                del fullName[0]
            
#             print ('http://engineering.iit.edu' + person.a['href'])
            # Opens and reads individual faculty pages
            personPage = urllib.request.urlopen('http://engineering.iit.edu' + person.a['href'])
            personSoup = BeautifulSoup(personPage.read(), 'lxml')
            
            # Locates and scrapes faculty positions
            positions = personSoup.find_all('div', class_='field-item')
#             print(fullName[0]+ ' ' + fullName[-1] + ': ' + positions[0].getText() + ' http://engineering.iit.edu' + person.a['href'])

            # Locates and scrapes phd information (university and year)
            education = personSoup.find_all('section', class_='field-name-field-education')
            for edu in education:
                phdSchool = ''
                phdYear = ''
                e= edu.div.div.getText()
                # Rules to help clean universities who could not be found by regex below
                if 'Ph.D.' in e:
                    if 'University of California, Los Angeles' in e:
                        phdSchool = "University of California-Los Angeles"
                    elif 'University of California, Berkeley' in e:
                        phdSchool = "University of California-Berkeley"
                    elif 'University of Illinois, Chicago' in e:
                        phdSchool = "University of Illinois-Chicago"
                    elif 'University of Illinois, Urbana' in e:
                        phdSchool = "University of Illinois-Urbana"
                    elif 'University of Wisconsin, Madison' in e:
                        phdSchool = "University of Wisconsin-Madison"
                    elif 'University of Illinois, Champaign' in e:
                        phdSchool = "University of Illinois Urbana-Champaign"
                    elif 'University of California, San Diego' in e:
                        phdSchool = "University of California - San Diego"
                    elif 'Texas A & M' in e:
                        phdSchool = "Texas A&M University"
                    elif 'University of Illinois at Chicago' in e:
                        phdSchool = "University of Illinois-Chicago"
                    elif 'Illinois Institute of Technology' in e:
                        phdSchool = "Illinois Institute of Technology"
                    elif 'Purdue University' in e:
                        phdSchool = "Purdue University"
                    elif 'University of Washington' in e:
                        phdSchool = "University of Washington"
                    elif 'Pennsylvania State University' in e:
                        phdSchool = "Pennsylvania State University"
                    elif 'Univ. of Calif. Berkeley' in e:
                        phdSchool = "University of California-Berkeley"
                    elif 'University of Michigan' in e:
                        phdSchool = "University of Michigan"
                    elif 'Northwestern University' in e:
                        phdSchool = "Northwestern University"
                    else:
                        # Finds university names
                        regex = r"(([A-Z][']?[^\s,.]+[.]?\s[(]?)*([a-z]\s)?(College|University|Institute|Law School|School of|Military Academy|Naval Academy|Nursing School|Seminary)[^,\d]*(?=,|\d))"
#                         print(e)
                        xString = re.split(r'[(]',e)[0]
                        resultSchool = re.search(regex, xString)
                        if resultSchool:
                            phdSchool = resultSchool.group(0)
                    
#                     print(phdSchool)
#                     print(fullName[0] + ' ' + fullName[-1])
#                     print(e)
                    
                    result = re.search(r'\d+', e)
                    if result:
                        phdYear = result.group(0)
#                         print('year: ' + result.group(0))
                    
                    # Appends information to lists to be used to create dataframe
                    firstNames.append(fullName[0].upper())
                    lastNames.append(fullName[-1].upper())
                    facultyUniversity.append(u)
                    facultyUniversityID.append(uID)
                    facultyDepartments.append(uDepartment)
                    jobTitles.append(positions[0].getText())
                    phdUniversity.append("" + phdSchool)
                    phdYears.append(phdYear)

In [5]:
# Create dataframe
data = pd.DataFrame({'First Name': firstNames, 
                     'Lastname': lastNames, 
                     'university of PhD': phdUniversity, 
#                      'university of PhD ID': universityID,
#                      'department of PhD': phdDepartments,
                     'year of PhD': phdYears,
                     'current university': facultyUniversity,
                     'current university ID': facultyUniversityID,
                     'department of current faculty': facultyDepartments,
#                      'year faculty started': facultyYearStarted,
                     'job title': jobTitles
                    })

# Orders columns of dataframe
data = data [['First Name', 'Lastname', 'university of PhD', 'year of PhD', 'current university', 'current university ID', 'department of current faculty', 'job title']]

In [6]:
# Creates empty columns for unrequired columns
data['year faculty started']=""
data['department of PhD']=""

In [7]:
# Re-order columns of dataframe
data = data[['First Name', 'Lastname', 'university of PhD', 'department of PhD', 'year of PhD','current university','current university ID','department of current faculty','year faculty started','job title']]

In [8]:
# data

In [9]:
# Reads past data csvs to compare university IDs
dfSchoolCodes = pd.read_csv('INFO370 - Assignment #1 - school_codes.csv')
dfFullOne = pd.read_csv('INFO370 - Assignment #1 - Full Data1.csv')
dfFullTwo = pd.read_csv('INFO370 - Assignment #1 - Full Data2.csv')
# df1 = dfFullOne.set_index('ugradschool')
df1 = dfFullOne['ugradschool'].unique()
df1 = dfFullOne.set_index('ugradschool')
# dfFullOne


In [10]:
# Compares university names and gathers IDs
my_uni_names = set(phdUniversity)
# print(my_uni_names)
ids = dfFullOne[dfFullOne['ugradschool'].isin(my_uni_names)][['ugradschoolID', 'ugradschool']].drop_duplicates()
# dfFullOne[dfFullOne['ugradschool'].isin(my_uni_names)]

# Cleans and rearanges dataframe
result = data.merge(ids, how='left', left_on='university of PhD', right_on='ugradschool')
result = result.drop('university of PhD', 1)
result = result.rename(columns={'ugradschool':'university of PhD', 'ugradschoolID':'university of PhD ID'})
result = result[['First Name', 'Lastname', 'university of PhD', 'university of PhD ID', 'department of PhD', 'year of PhD','current university','current university ID','department of current faculty','year faculty started','job title']]
result = result[np.isfinite(result['university of PhD ID'])]
result = result.drop_duplicates(subset=['First Name', 'Lastname'], keep='last')

# Creates new csv
result.to_csv('Ho_Full_Data.csv')
result

,First Name,Lastname,university of PhD,university of PhD ID,department of PhD,year of PhD,current university,current university ID,department of current faculty,year faculty started,job title
0,JAN,LEUCHTENBERGER,University of Michigan,108.0,,2005,University of Puget Sound,365,Asian Languages & Cultures,,Associate Professor and Director
1,MENGJUN,LI,The Ohio State University,172.0,,2014,University of Puget Sound,365,Asian Languages & Cultures,,Assistant Professor
4,WILLIAM,BARRY,University of Michigan,108.0,,1988,University of Puget Sound,365,Classics,,Professor
5,AISLINN,MELCHIOR,University of Pennsylvania,144.0,,2004,University of Puget Sound,365,Classics,,Professor
6,ERIC,ORLIN,University of California-Berkeley,1.0,,1994,University of Puget Sound,365,Classics,,Professor and Chair
8,MEGAN,DANIELS,Stanford University,232.0,,2016,University of Puget Sound,365,Classics,,Lora Bryning Redford Post-Doctoral Fellow in A...
9,RACHEL,DEMOTTS,University of Wisconsin-Madison,16.0,,2005,University of Puget Sound,365,Environmental Policy & Decision Making,,Associate Professor
10,LEA,FORTMANN,The Ohio State University,172.0,,2014,University of Puget Sound,365,Environmental Policy & Decision Making,,Assistant Professor
11,KENA,FOX-DOBBS,University of California-Santa Cruz,15.0,,2006,University of Puget Sound,365,Environmental Policy & Decision Making,,Associate Professor
12,PETER,HODUM,University of California-Davis,6.0,,1999,University of Puget Sound,365,Environmental Policy & Decision Making,,Associate Professor
